### Neural Networks

In [1]:
import os
import glob
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, cross_val_score
from tensorflow import set_random_seed

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import ModelCheckpoint

from constants import DATA_DIR
from utils import csv_concatenate, calculate_FPTS, calculate_MAE, calculate_RMSE, cross_val, load_full_dataset

Using TensorFlow backend.


In [2]:
np.random.seed(23)
set_random_seed(23)

warnings.filterwarnings("ignore")

In [29]:
def root_mean_squared_error(y_true, y_pred):
    # Custom loss function for keras
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

def get_modelcheckpoint_path(model_num):
    #Create a file path for a model and save models in hdf5 files with datetime, validation losses and epochs
    parent = DATA_DIR+"/Models/NN/Model_{}/".format(model_num)
    child = pd.Timestamp.now().strftime('%Y%m%d-%Hh%Mm') + "-model-epoch_{epoch:02d}-rmse_{val_loss:.5f}.hdf5"
    return parent + child 

def create_model(model_num):
    def model_1():
        model = Sequential()
        model.add(Dense(X.shape[1], input_dim=X.shape[1], activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(1))
        model.compile(loss=root_mean_squared_error, optimizer='adam')
        model.summary()
        return model

    def model_2():
        model = Sequential()
        model.add(Dense(X.shape[1], input_dim=X.shape[1], activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1))
        model.compile(loss=root_mean_squared_error, optimizer='adam')
        model.summary()
        return model

    def model_3():
        model = Sequential()
        model.add(Dense(X.shape[1], input_dim=X.shape[1], activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(256, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1))
        model.compile(loss=root_mean_squared_error, optimizer='adam')
        model.summary()
        return model
    
    if model_num == 1:
        return model_1
    elif model_num == 2:
        return model_2
    elif model_num == 3:
        return model_3
    else:
        return "invalid model_num"

In [33]:
X, y = load_full_dataset('quad')

In [ ]:
for i in range(1, 4):
    model = KerasRegressor(build_fn=create_model(i),
                           epochs=100,
                           batch_size=32,
                           validation_split=0.2,
                           shuffle=True,
                           verbose=1)

    filepath = get_modelcheckpoint_path(i)
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=False, verbose=0, mode='min')
    callbacks_list = [checkpoint]

    if i == 1:
        h1 = model.fit(X, y, callbacks=callbacks_list)
    elif i == 2:
        h2 = model.fit(X, y, callbacks=callbacks_list)
    elif i == 3:
        h3 = model.fit(X, y, callbacks=callbacks_list)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 41)                1722      
_________________________________________________________________
dense_46 (Dense)             (None, 64)                2688      
_________________________________________________________________
dense_47 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 33        
Total params: 6,523
Trainable params: 6,523
Non-trainable params: 0
_________________________________________________________________
Train on 96004 samples, validate on 24001 samples
Epoch 1/100
96004/96004 [==============================] - 12s 128us/step - loss: 9.4893 - val_loss: 9.4079
Epoch 2/100
96004/96004 [==============================] - 11s 116us/step - loss: 9.1607 - val_loss: 9.4308
Epoch 

In [ ]:
for i, hist in enumerate([h1, h2, h3]):
    plt.subplot(1,3,i)  
    plt.plot(hist.history['loss'])  
    plt.plot(hist.history['val_loss'])  
    plt.title('Model Loss')  
    plt.ylabel('Loss')  
    plt.xlabel('Epoch')  
    plt.legend(['Train', 'Validation'], loc='upper right')  
    plt.show()

In [ ]:
model = KerasRegressor(build_fn=model_1,
                       epochs=30,
                       batch_size=32,
                       validation_split=0.2,
                       shuffle=True,
                       verbose=0)

kfold = KFold(n_splits=5, shuffle=True)

In [ ]:
results_MAE = cross_val_score(model, X, y, cv=kfold, n_jobs=1, scoring='neg_mean_absolute_error')
results_RMSE = cross_val_score(model, X, y, cv=kfold, n_jobs=1, scoring='neg_mean_squared_error')

In [ ]:
print(np.sqrt(-results_RMSE))
print("Results: %.4f RMSE" % np.sqrt(np.mean(-results_RMSE)))

print(np.sqrt(-results_MAE))
print("Results: %.4f MAE" % np.mean(-results_MAE))

In [ ]:
model = KerasRegressor(build_fn=model_3,
                       epochs=15,
                       batch_size=32,
                       validation_split=0.2,
                       shuffle=True,
                       verbose=2)

kfold = KFold(n_splits=5, shuffle=True)

In [ ]:
results_MAE = cross_val_score(model, X, y, cv=kfold, n_jobs=1, scoring='neg_mean_absolute_error')
results_RMSE = cross_val_score(model, X, y, cv=kfold, n_jobs=1, scoring='neg_mean_squared_error')

In [ ]:
print(np.sqrt(-results_RMSE))
print("Results: %.4f RMSE" % np.sqrt(np.mean(-results_RMSE)))

print(np.sqrt(-results_MAE))
print("Results: %.4f MAE" % np.mean(-results_MAE))

### Prediction

In [ ]:
### Train Test Split
X = df_features.sort_values(by=['Date','Name']).reset_index(drop=True)

target_month = 201903

start = 20190301
end = 20190331

test_indices = (df_features['Date'] >= start) & (df_features['Date'] <= end)
train_indices = [not value for value in test_indices]

X_train = df_features.loc[train_indices, selected]
X_test = df_features.loc[test_indices, selected]

y_train = df_features.loc[train_indices, 'FPTS'].values.reshape(-1,1).flatten()
y_test = df_features.loc[test_indices, 'FPTS'].values.reshape(-1,1).flatten()

# X_train = MinMaxScaler().fit_transform(X_train)
# X_test = MinMaxScaler().fit_transform(X_test)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
pred_baseline = df_baseline.loc[(df_baseline['Date'] >= start) & (df_baseline['Date'] <= end), 'FPTS_pred'].reset_index(drop=True)
actual = df_baseline.loc[(df_baseline['Date'] >= start) & (df_baseline['Date'] <= end), 'FPTS'].reset_index(drop=True)

print(calculate_MAE(pred_baseline, actual))
print(calculate_RMSE(pred_baseline, actual))

In [ ]:
reg = LinearRegression()
reg.fit(X_train, y_train)
pred_lm = reg.predict(X_test)

print(calculate_MAE(pred_lm, y_test))
print(calculate_RMSE(pred_lm, y_test))

In [ ]:
d_train = lgb.Dataset(X_train, label=y_train)
d_test = lgb.Dataset(X_test, label=y_test)

watchlist = [d_valid]

opt_params = df_params.loc[0, 'params']

model = lgb.train(opt_params, d_train, watchlist, verbose_eval=1)
pred_gbm = model.predict(X_test)

print(calculate_MAE(pred_gbm, y_test))
print(calculate_RMSE(pred_gbm, y_test))

In [ ]:
best_parameters = {'max_depth':6, 'n_estimators':250, 'min_child_weight':4, 'colsample_bytree':0.6, 
                   'colsample_bylevel':0.7, 'subsample':1.0, 'gamma':0.0, 'learning_rate':0.026944654231987667}

reg = xgb.XGBRegressor(**best_parameters)
reg.fit(X_train, y_train, verbose=1)
pred_xgb = reg.predict(X_test)

print(calculate_MAE(pred_xgb, y_test))
print(calculate_RMSE(pred_xgb, y_test))

In [ ]:
def advanced_model():
    model = Sequential()
    model.add(Dense(X_train.shape[1], input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
model = KerasRegressor(build_fn=advanced_model,
                       epochs=30,
                       batch_size=64,
                       validation_split=0.2,
                       shuffle=True,
                       verbose=1)

h = model.fit(X_train, y_train)

In [ ]:
pred_nn = model.predict(X_test)
print(calculate_MAE(pred_nn, y_test))
print(calculate_RMSE(pred_nn, y_test))

### Write prediction into csv

In [ ]:
df_pred = df_features.loc[test_indices, ['Date', 'Name', 'Team', 'FPTS', 'Pos', 'Salary']]
df_pred['Pred'] = pred_gbm

In [ ]:
df_pred.to_csv(os.path.join(DATA_DIR, 'Predictions/{}.csv'.format(pd.Timestamp.now().strftime('%Y%m%d-%Hh%Mm'))), index=False)